https://azad-wolf.medium.com/latent-space-representation-a-hands-on-tutorial-on-autoencoders-in-tensorflow-57735a1c0f3f

In [ ]:
!pip install tensorflow

In [10]:
!pwd

!ls -U //kaggle/input/fashion-product-images-small/images | head -4


/kaggle/working
31973.jpg
30778.jpg
19812.jpg
22735.jpg
ls: write error: Broken pipe


In [4]:
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib


## create train and validation datasets
DB_PATH = "//kaggle//input//fashion-product-images-small//images//"
BUFFER_SIZE = 10000
BATCH_SIZE = 1000
IMG_WIDTH = 60
IMG_HEIGHT = 60

def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image,channels=3)

    input_image = tf.cast(image, tf.float32)
    return input_image

def random_crop(input_image):
    cropped_image = tf.image.random_crop(
      input_image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image

def resize(input_image):
    input_image = tf.image.resize(input_image, [IMG_HEIGHT, IMG_WIDTH],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return input_image

def normalize(input_image):
    input_image = (input_image / 255)
    return input_image

@tf.function()
def random_jitter(input_image):
    input_image = random_crop(input_image)
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        
    return input_image

def load_image_train(image_file):
    input_image = load(image_file)
    #input_image = random_jitter(input_image)
    input_image = resize(input_image)
    input_image = normalize(input_image)

    return input_image,input_image

def load_image_test(image_file):
    input_image = load(image_file)
    #input_image = random_jitter(input_image)
    input_image = resize(input_image)
    input_image = normalize(input_image)
    return input_image,input_image


data_dir = pathlib.Path(DB_PATH)
image_count = len(list(data_dir.glob('*.jpg')))
print(f"image count {image_count}")
dataset = tf.data.Dataset.list_files(DB_PATH)


val_size = int(image_count * 0.2)
train_ds = dataset.skip(val_size)
val_ds = dataset.take(val_size)

print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())


train_ds = train_ds.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_ds = val_ds.map(load_image_test)
val_ds = val_ds.batch(BATCH_SIZE)



image count 44441
0
1


In [3]:
dataset = tf.data.Dataset.list_files(DB_PATH)

val_size = int(image_count * 0.2)
train_ds = dataset.skip(val_size)
val_ds = dataset.take(val_size)

train_ds.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [8]:
tf.data.Dataset.list_files(DB_PATH+"*.jpg")

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: //kaggle//input//fashion-product-images-small//images//*.jpg'

In [8]:
val_ds.take(1)

<_TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [5]:
list(val_ds.as_numpy_iterator())[0]

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /kaggle/input/fashion-product-images-small/images; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [4]:
plt.figure(figsize=(15, 5))
for images,targets in val_ds.take(1):
    for i in range(45):
      ax = plt.subplot(3, 15, i + 1)
      plt.imshow(images[i].numpy().astype("float32"))
      plt.axis("off")

TypeError: Cannot iterate over a scalar tensor.

<Figure size 1500x500 with 0 Axes>